## Section create one geojson for municipalities

In [ ]:
from os import listdir
import json

files=pd.Series(listdir('../Mexican-Municipalities/')) ## https://github.com/elbic/Mexican-Municipalities
files=foo[foo.str.contains('geojson')]

In [ ]:
buffer={'type': 'FeatureCollection', 'features':[]}

for i in files:
    with open('../Mexican-Municipalities/'+i) as json_file:
        geojson = json.load(json_file)
        buffer['features'].append(geojson['features'][0])

In [ ]:
with open('municip.json', 'w') as fp:
    json.dump(buffer, fp)

In [ ]:
import json
with open('municip.json') as f:
    buffer=json.load(f)

In [ ]:
import pandas as pd
df=pd.DataFrame([i['properties'] for i in buffer['features']])

In [ ]:
buffer1=buffer
buffer1['features']=buffer1['features'][:20]

In [ ]:
import plotly.express as px
fig = px.choropleth_mapbox(df[:20], geojson=buffer1, color='CVE_MUN',
                           locations="concat", featureidkey="properties.concat",
                           mapbox_style="carto-positron", zoom=3)
fig.show()

# Section of centroids

In [ ]:
import json
with open('municip.json') as f:
    buffer=json.load(f)

In [ ]:
from shapely.geometry import shape, GeometryCollection
coll=GeometryCollection([shape(i['geometry']) for i in buffer['features']])

In [ ]:
centroids=[list(i.centroid.coords)[0] for i in coll]

In [ ]:
for i in range(len(buffer['features'])):
    buffer['features'][i]['centroid']=centroids[i]

In [ ]:
with open('municip.json', 'w') as fp:
    json.dump(buffer, fp)

## Section to smooth edges

In [ ]:
simplified=coll.simplify(0.01, preserve_topology=True)

In [ ]:
from collections import Iterable
for i in range(len(simplified)):
#for i in range(12):
    #print(i)
    foo=[]
    if isinstance(simplified[i], Iterable):
        for j in simplified[i]:
            foo1=[]
            for k in j.exterior.coords:
                foo1.append(list(k))
            foo.append([foo1])
    else:
        foo1=[]
        for k in simplified[i].exterior.coords:
                foo1.append(list(k))
        foo.append(foo1)
    #print(foo)
    buffer['features'][i]['geometry']['coordinates']=foo

In [ ]:
with open('municip_simplified.json', 'w') as fp:
    json.dump(buffer, fp)

## Check results

In [ ]:
import json
with open('municip_simplified.json') as f:
    muni_simp=json.load(f)

In [ ]:
muni_simp['features'][1]

In [ ]:
muni['features'][1]['geometry']

In [ ]:
import pandas as pd
df=pd.DataFrame([i['properties'] for i in muni_simp['features']])

In [ ]:
buffer1=muni_simp.copy()
buffer1['features']=buffer1['features'][:500]

In [ ]:
df.reset_index(inplace=True)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=muni_simp, featureidkey='properties.concat', locations=df.concat, z=df.index,
                                    colorscale="Viridis",
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()